**DATASET - selezione e importazione**
---

WASHINGTON
1.  EV - Population (https://catalog.data.gov/dataset/electric-vehicle-population-data)
2. EV - Adoption Rate (https://data.wa.gov/Transportation/Monthly-Electric-Vehicle-Adoption-Rate-by-County/crrp-awfs
)

ALTRI
1.   Connecticut EV - Chiarging Stations (https://catalog.data.gov/dataset/electric-vehicle-charging-stations)
2.   Dataset strano (https://catalog.data.gov/dataset/motor-vehicle-registrations-dashboard-data)


In [1]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output


pop = pd.read_csv(r"C:\ARD_Dataset\Electric_Vehicle_Population_Data.csv")
adopt_date_pulite = pd.read_excel(r"C:\ARD_Dataset\Electric_Vehicle_Population_Size_History_By_County.xlsx")
df3 = pd.read_csv(r"C:\ARD_Dataset\DATASET_ESAME_date_pulite-_1_.csv")
df4 = pd.read_excel(r"C:\ARD_Dataset\DATASET_ESAME_date_pulite.xlsx")

**PULIZIA dei DATASET**
---


> **pop**

1. eliminare righe che non hanno WA, e poi tutta la colonna stato
2. eliminare colonna VIN
3. eliminare Electric Utility
4. eliminare 2020 Censun Tract
5. eliminare Base MRP
6. eliminare Electric Range (utile ai fini dell'analisi dello stato?)
7. Postal Code serve per la mappa?



> **adopt**

1. eliminare colonna stato WA
2. eliminare righe Vehicle Primary Use = "truck"


In [2]:

#Rimozione colonne futili

pop.drop('VIN (1-10)', axis=1, inplace=True)
pop.drop('Electric Utility', axis=1, inplace=True)
pop.drop('2020 Census Tract', axis=1, inplace=True)
pop.drop('Base MSRP', axis=1, inplace=True)
pop.drop('Electric Range', axis=1, inplace=True)
pop.drop('Clean Alternative Fuel Vehicle (CAFV) Eligibility', axis=1, inplace=True)
pop.drop('Legislative District', axis=1, inplace=True)
pop.drop('DOL Vehicle ID', axis=1, inplace=True)
pop.drop('Vehicle Location', axis=1, inplace=True)

#Rinmina colonne

pop.rename(columns = {'Postal Code':'Postal_Code'}, inplace = True)
pop.rename(columns = {'Model Year':'Model_Year'}, inplace = True)
pop.rename(columns = {'Electric Vehicle Type':'EV_Type'}, inplace = True)
pop.rename(columns = {'Vehicle Location':'Vehicle_Location'}, inplace = True)


pop_tmt = pop[pop.State == "WA"]

In [3]:

adopt_date_pulite.rename(columns = {'Vehicle Primary Use':'User'}, inplace = True)
adopt_date_pulite.rename(columns = {'Battery Electric Vehicles (BEVs)':'Battery_EV'}, inplace = True)
adopt_date_pulite.rename(columns = {'Plug-In Hybrid Electric Vehicles (PHEVs)':'Plugin_Hybrid_EV'}, inplace = True)
adopt_date_pulite.rename(columns = {'Electric Vehicle (EV) Total':'EV_Total'}, inplace = True)
adopt_date_pulite.rename(columns = {'Non-Electric Vehicle Total':'Non_EV_Total'}, inplace = True)
adopt_date_pulite.rename(columns = {'Total Vehicles':'Vehicles_Total'}, inplace = True)
adopt_date_pulite.rename(columns = {'Percent Electric Vehicles':'Percent_EV'}, inplace = True)
adopt_date_pulite[adopt_date_pulite.User == "Passenger"]


,Date,County,State,User,Battery_EV,Plugin_Hybrid_EV,EV_Total,Non_EV_Total,Vehicles_Total,Percent_EV
1,January 2017,Adams,WA,Passenger,1,1,2,12175,12177,0.02
3,January 2017,Asotin,WA,Passenger,4,3,7,12321,12328,0.06
5,January 2017,Benton,WA,Passenger,129,129,258,127871,128129,0.20
7,January 2017,Chelan,WA,Passenger,54,28,82,51904,51986,0.16
9,January 2017,Clallam,WA,Passenger,73,64,137,48010,48147,0.28
...,...,...,...,...,...,...,...,...,...,...
5606,December 2022,Wahkiakum,WA,Passenger,26,12,38,2939,2977,1.28
5608,December 2022,Walla Walla,WA,Passenger,195,106,301,34669,34970,0.86
5610,December 2022,Whatcom,WA,Passenger,2043,772,2815,143938,146753,1.92
5612,December 2022,Whitman,WA,Passenger,109,64,173,21308,21481,0.81


In [4]:
##Piatte Github - sabato sera - tentativo Dash non verticale
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output
import json
import requests
import plotly.express as px
import geopandas as gpd
from decimal import Decimal, ROUND_DOWN



#Codice per Grafico 2
pop1 = pop
Makers = np.append(pop_tmt.Make.unique(), '-default- Tutti')
County = np.append(pop_tmt.County.unique(), '-default- Tutte')


app_Graph_Totale = JupyterDash(__name__)

app_Graph_Totale.layout = html.Div(style={'backgroundColor': '#FDFFB2', 'position':'absolute', 'top':0, 'bottom':0, 'right':0, 'left':0, 'font-family':'Bahnschrift'}, children=[
  html.H1("Electric Vehicle Washington State", style= {'text-align' : 'center'}),
  html.Div([

    #PRIMO GRAFICO 
      html.Div([dcc.Markdown(children = 'Seleziona la **data di inizio**',    
                             style = {'position': 'absolute', 'margin-left': '1%', 'margin-top': '34%'}),
                dcc.Dropdown(id = 'dropdown_start_input',
                             options = df3.Date.unique(),
                             value = 'January  2017',
                             style = {'position': 'absolute', 'margin-left': '1%', 'margin-top': '28.5%', 'width' : '30%'}),
                dcc.Markdown(children = "Seleziona la **data di fine**",
                             style = {'width' : '100%', 'position': 'absolute', 'margin-left': '1%', 'margin-top': '41%'}),
                dcc.Dropdown(id = 'dropdown_end_input',
                             options = df3.Date.unique(),
                             value = 'December  2022',
                             style = {'position': 'absolute', 'margin-left': '1%', 'margin-top': '34%', 'width' : '30%'}),
                dcc.Graph(id = 'figure_output_1',
                          style = {'width' : '80%', 'position' : 'absolute', 'margin-left' : '15%', 'margin-top': '26%'}),
              ]), 



          
    #SECONDO GRAFICO   
      html.Div([
                dcc.Graph(id = 'figure_output_2',
                             style = {'position' : 'absolute', 'width' : '36%', 'margin-left' : '10%', 'margin-top': '0%'}),
                dcc.Markdown(children = 'Seleziona il **marchio**',    
                             style = {'position' : 'absolute','margin-left' : '1%', 'margin-top': '5%', 'display':'block'}),
                dcc.Dropdown(id = 'dropdown_marchio',
                             options = sorted(Makers),
                             value = '-default- Tutti',
                             style = {'position' : 'absolute', 'margin-left' : '1%', 'margin-top': '6%', 'width':'29%'}),
                dcc.Markdown(children = "Seleziona la **contea**",
                             style = {'position' : 'absolute', 'margin-left' : '1%', 'margin-top': '12%'}),
                dcc.Dropdown(id = 'dropdown_contea',
                             options = sorted(County),
                             value = '-default- Tutte',
                             style = {'position' : 'absolute', 'margin-left' : '1%', 'margin-top': '12%', 'width':'29%'}),
              ]),


  #TERZO GRAFICO 
      html.Div([dcc.Markdown(children = 'Seleziona la **data**',
                             style = {'position' : 'absolute', 'margin-left' : '91%', 'margin-top' : '4%'}),    
                dcc.Dropdown(id = 'dropdown_start_input_Graph3',
                             options = adopt_date_pulite.Date.unique(),
                             value = 'January  2017',
                             style = {'position' : 'absolute', 'margin-left' : '71%', 'margin-top' : '6%', 'width' : '28%'}),    
                dcc.Markdown(children = 'Seleziona il **tipo**',
                             style = {'position' : 'absolute', 'margin-left' : '91%', 'margin-top' : '11%'}),    
                dcc.Dropdown(id = 'dropdown_type_input',
                             options = ['-default- Qualsiasi', 'Battery_EV' , 'Plugin_Hybrid_EV'],
                             value = '-default- Qualsiasi',
                             style = {'position' : 'absolute', 'margin-left' : '71%', 'margin-top' : '11%', 'width' : '28%'}),    
                dcc.Graph(id = 'figure_output_3',
                          style = {'position':'absolute', 'width' : '45%', 'margin-left' : '45%', 'margin-top' : '0%'}),  
              ])
  
  ])])

app_Graph_Totale.run_server(mode = 'external')

#   #CALLBACK PRIMO GRAFICO 

@app_Graph_Totale.callback(Output(component_id = 'figure_output_1', component_property = 'figure'),
            Input(component_id = 'dropdown_start_input', component_property = 'value'),
            Input(component_id = 'dropdown_end_input', component_property = 'value'))

def update_figure_output(start,end):
  df5 = df4
  pd.to_datetime(df5['Date'])
  df6 = df5.Date.between(start,end)
  df6 = df5.loc[df6]

  fig0 = px.line(df6, y='EV_Total', x='Date', color = 'ID_Colore_Totale', color_discrete_map= {'Totale' : 'red'}, hover_data = {'ID_Colore_Totale' : False, 'Date' : False})
  fig1 = px.bar(df6, y='Plugin_Hybrid_EV', x='Date',
                color = 'ID_Colore_Plugin', color_discrete_map= {'Ibrido plugin' : '#3366CC'}, labels={'Plugin_Hybrid_EV':'Plugin_Hybrid_EV Total'}, hover_data = {'ID_Colore_Plugin' : False, 'Date' : False})
  fig2 = px.bar(df6, y='Battery_EV', x='Date',
                color = 'ID_Colore_Batteria', color_discrete_map= {'Batteria' : "#0099c6"}, 
                labels={'Battery_EV':'Battery_EV Total'},  hover_data = {'ID_Colore_Batteria' : False, 'Date' : False})
  fig = go.Figure(data = fig1.data + fig2.data + fig0.data)
  fig.update_layout(barmode="stack", 
                    legend_traceorder= 'reversed',
                    transition = {'duration': 1000}, 
                    hovermode="x unified",
                    title= {'text':'Andamento temporale del mercato di auto elettriche nello stato di Washington',
                            'y': 0.83,
                            'x': 0.5,
                            'xanchor':'center',
                            'yanchor':'top'},
                    height = 400,
                    paper_bgcolor = '#FDFFB2',
                    plot_bgcolor = 'white',
                    title_font=dict(family="Bahnschrift"))                           
  # )
  #  clickmode="event+select"
  return fig





#CALLBACK SECONDO GRAFICO

@app_Graph_Totale.callback(Output(component_id = 'figure_output_2', component_property = 'figure'),
                           Input(component_id = 'dropdown_marchio', component_property = 'value'),
                           Input(component_id = 'dropdown_contea', component_property = 'value'))

def pix_pie (make, county):
  if (county == '-default- Tutte' and make == '-default- Tutti'): 
    test = pop_tmt
  elif(county == '-default- Tutte'): 
    test = pop_tmt[pop_tmt['Make'] == make]
  elif(make == '-default- Tutti'):
    test = pop_tmt[pop_tmt['County'] == county]
  else:
    test = pop_tmt[pop_tmt['County'] == county]
    test = test[test['Make'] == make]

  counts = test['Model'].value_counts().reset_index()
  counts.columns = ['Model', 'Number_of_Vehicles']
  total_count = counts['Number_of_Vehicles'].sum()
  counts['percent_of_total'] = counts['Number_of_Vehicles'] / total_count * 100
  counts['percent_of_total'] = counts['percent_of_total'].round(decimals = 2)

  #Rinomino tutti i modelli con percentuale minore di 2 con others, analogo al sito di Washington
  counts.loc[counts['percent_of_total'] < 2, 'Model'] = "Others"

  #Plotto la tortina
  fig = px.pie(counts, values = 'Number_of_Vehicles' , names = 'Model', 
               hover_data=['percent_of_total'], 
               labels={'percent_of_total':'Percent of total','Number_of_Vehicles':'Number of vehicles'})
  fig.update_layout(title= {'text':'Percentuale dei vari modelli nelle varie contee',
                            'y': 1,
                            'x': 0.5,
                            'xanchor':'center',
                            'yanchor':'top'},
                    paper_bgcolor = '#FDFFB2',
                    plot_bgcolor = 'white',
                    title_font=dict(family="Bahnschrift")
                    )                             
                    
  return fig


#CALLABCK TERZO GRAFICO 


@app_Graph_Totale.callback(Output(component_id = 'figure_output_3', component_property = 'figure'),
            Input(component_id = 'dropdown_start_input_Graph3', component_property = 'value'),
            Input(component_id = 'dropdown_type_input', component_property = 'value'))
            

def update_figure_output(start,type):

  # # URL for the raw JSON file on Github
  url='https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'

  # # Make a GET request to the URL
  response = requests.get(url)

  # # Check if the request was successful
  if response.status_code == 200:
    # Load the JSON data from the response
    data = json.loads(response.content)
    # Do something with the data...
  else:
    # Handle the error
      print("Failed to download data. Status code:", response.status_code)

  # Filter only the counties of Washington
  filtered_data = {"type": "FeatureCollection", "features": []}
  for feature in data["features"]:
      if feature["properties"]["STATE"] == "53":
          filtered_data["features"].append(feature)

  #Calculate percantage of Full_EV and Hybrid_EV
  tmp = adopt_date_pulite[adopt_date_pulite.Date == start]
  tmp1 = tmp[tmp.User == "Passenger"]
  
  if (type == '-default- Qualsiasi'): 
    type = 'EV_Total'
  
  tmp1['Percent_value'] = tmp1[type] / tmp1['Vehicles_Total'] *100
  tmp1['Percent_value'] = tmp1['Percent_value'].round(decimals = 2)
  

  fig = px.choropleth(tmp1, geojson = filtered_data, 
                      featureidkey= "properties.NAME",
                      locations= "County",
                      color = 'Percent_value',
                      hover_name= "County",
                      hover_data = {type:True, 'Non_EV_Total':True, 'County' : False},
                      labels={'Percent_value': type , type : type + ' Total'},
                      # range_color= (0,median),
                      basemap_visible = True,
                      fitbounds = "locations",   
                      color_continuous_scale = 'Blues'
                      )
  fig.update_layout(title = {'text' : type + ' percent over Total Vehicles for every county',
                      'y': 0.9,
                      'x': 0.5,
                      'xanchor':'center',
                      'yanchor':'top'},
                      paper_bgcolor = '#FDFFB2',
                      plot_bgcolor = 'white',
                      title_font=dict(family="Bahnschrift")
                      ),                             
  # )
  return fig

Dash app running on http://127.0.0.1:8050/


C:\Users\andre\AppData\Local\Temp\ipykernel_19480\3761652994.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\andre\AppData\Local\Temp\ipykernel_19480\3761652994.py:206: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

